<a href="https://colab.research.google.com/github/RenanCostaNascimento/mestrado-reconhecimento-padroes/blob/main/Avaliando_a_generaliza%C3%A7%C3%A3o_de_algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O objetivo deste Notebook é utilizar duas formas de avaliação para comparar classificadores e verificar qual é a melhor forma. Cada forma se avaliação será executada 10 vezes, e a média dos resultados indicará os resultados. Os classificadores escolhidos são o [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) e o [KNN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) do scikit-learn. Já as formas de avaliação serão: usar um etapa de processamento (StandardScale) da base toda, e usar uma combinação de GridSearch, Pipeline e StandardScale. Vamos começar importando a base de dados [Heart Disease UCI do Kaggle](https://www.kaggle.com/ronitf/heart-disease-uci). É um dataset que indica se a pessoa tem alguma condição cardíaca com base nas suas informações médicas.


In [1]:
import pandas as pd

dataset = pd.read_csv('/content/heart.csv')
dataset.head(3)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1


Os dados da base de dados devem ser pré-processados. Para tanto, vamos começar separando os dados em uma matriz de features (dados de entrada) e um vetor coluna com as classes (última coluna – “target” – do arquivo csv).

In [2]:
import numpy as np

# transforma o dataset do pandas em np array
featureMatrix = dataset.to_numpy()

# extrai a ultima coluna da matrix
labels = featureMatrix[:,len(featureMatrix[0]) - 1]

# remove a ultima coluna da featureMatrix
featureMatrix = np.delete(featureMatrix, len(featureMatrix[0]) - 1, 1)

np.shape(featureMatrix), np.shape(labels)

((303, 13), (303,))

Vamos agora para a primeira forma de avaliação. Vamos começar usando uma etapa de pré-processamento ([Standard Scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)) da base inteira. Em seguida precisamos dividir nossa base de dados em treino e teste. Para fazer isso vamos utilizar a função [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) do sklearn. Por fim, vamos de fato treinar nossos modelos. Para garantir que a comparação dos classificadores seja feita da melhor forma possível, vamos treinar o modelo de 10 formas diferentes, cada uma com um random_state único. Isso nos dará 20 modelos distintos, um para cada classificador. Para facilitar o trabalho, abaixo definimos uma função que separa a base de dados em teste e treino, além de treinar os modelos.

In [41]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

scaler = StandardScaler()
standardFeatures = scaler.fit_transform(featureMatrix)

def trainTestSplit(randomState):
  return train_test_split(standardFeatures, labels, test_size=0.33, random_state=randomState)

def trainModels(featuresTrain, labelsTrain):
  # eu precisei colocar o max_iter=1000 porque estava dando esse erro sem ele:
  #  ConvergenceWarning: lbfgs failed to converge (status=1): STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
  logisticModel = LogisticRegression(max_iter=10000)
  logisticModel.fit(featuresTrain, labelsTrain)

  knnModel = KNeighborsClassifier()
  knnModel.fit(featuresTrain, labelsTrain)

  return logisticModel, knnModel

def evaluateModel(model, featuresTest, labelsTest):
  prediction = model.predict(featuresTest)
  return f1_score(labelsTest, prediction)

logisticScores1 = []
knnScores1 = []

for test in range(10):
  featuresTrain, featuresTest, labelsTrain, labelsTest = trainTestSplit(test)
  logisticModel, knnModel = trainModels(featuresTrain, labelsTrain)

  logisticF1Score = evaluateModel(logisticModel, featuresTest, labelsTest)
  logisticScores1.append(logisticF1Score)

  knnF1Score = evaluateModel(knnModel, featuresTest, labelsTest)
  knnScores1.append(knnF1Score)

print("Evaluation 1")
print(f"Logistic F1 Score: {np.mean(logisticScores1)}")
print(f"KNN F1 Score: {np.mean(knnScores1)}")

Evaluation 1
Logistic F1 Score: 0.8496124344774874
KNN F1 Score: 0.8385658681341935


Na segunda forma de avaliação também vamos usar o StandardScaler, mas dessa vez vamos passá-lo dentro de um Pipeline. Além disso, usaremos o [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) para encontrar os hiperparâmetros dos estimadores automaticamente. Por fim, usaremos o [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) para treinar. Para treinar os modelos 10 vezes, vamos usar o [RepeatedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedKFold.html) para garantir que em toda rodada os dois estimadores sejam treinados com a mesma configuração do fold.

In [55]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedKFold

logisticPipeline = Pipeline([
  ("standardization", scaler),
  ("classifier", LogisticRegression(max_iter=10000))
])
logisticParams = { "classifier__penalty": ["l2", "none"] }
logisticModel = GridSearchCV(logisticPipeline, logisticParams, scoring="f1")

knnPipeline = Pipeline([
  ("standardization", scaler),
  ("classifier", KNeighborsClassifier())
])
knnParams = { "classifier__n_neighbors": [3, 5, 7] }
knnModel = GridSearchCV(knnPipeline, knnParams, scoring="f1")

logisticScores2 = []
knnScores2 = []

for test in range(10):
  repeatedKFold = RepeatedKFold(random_state=test)

  logisticScore = cross_validate(logisticModel, featureMatrix, labels, scoring="f1", cv=repeatedKFold)
  logisticScores2.append(np.mean(logisticScore["test_score"]))

  knnScore = cross_validate(knnModel, featureMatrix, labels, scoring="f1", cv=repeatedKFold)
  knnScores2.append(np.mean(knnScore["test_score"]))

print("Evaluation 2")
print(f"Logistic F1 Score: {np.mean(logisticScores2)}")
print(f"KNN F1 Score: {np.mean(knnScores2)}")

Evaluation 2
Logistic F1 Score: 0.8448686538806449
KNN F1 Score: 0.838742082369803
